In [1]:
import pandas as pd, json, numpy as np
from scanr_utils import *
from casuhal_utils import *
# auto reload modules
%load_ext autoreload
%autoreload 2

## 1. Récupérer les DOI

In [2]:
## option A : utiliser un fichier local 

utiliser_fichier_local = True

if utiliser_fichier_local : 
    ## un fichier local placé dans le dossier *data*
    ## doit être un fichier .csv avec encodage utf8 et ',' comme séparateur
    ## avec au moins une colonne *doi* 
    ## precier le nom de cette colonne (DOI, doiId_s etc. )
    doi_col_name = "DOI"
    
    df_mydois = pd.read_csv("data/2023-05-halathon-scopus-sante.csv", usecols = [doi_col_name])
    
    # dans la suite du code la colonne doit être nommée doi
    df_mydois.rename(columns = {doi_col_name : "doi"}, inplace = True)
    
    ## limiter le nombre de ligne/publication
    df_mydois = df_mydois[:400]
    print(f"nb de DOI importés depuis fichier \t{len(df_mydois)}")
else : 
    df_mydois = pd.DataFrame()

nb de DOI importés depuis fichier 	400


In [3]:
## option B : utiliser ScanR
# les deux options peuvent être valides, un dédoublonnage sera alors effectué

utiliser_scanR = False # True pour oui False pour non 

if utiliser_scanR : 
    
    id_etab = "180089047" # identifiant UAI de votre établissement
    # UP 0755976N, Nantes 194409843, INRIA 180089047
    min_year = 2020 # l'année à partir de laquelle commencer la récupération. 
    # scanR inclut les publications jusqu'a 2020.
    # 2020 donne les publications de 2020. 2019 donne les publications de 2019 et 2020 etc.

    df_scanr = get_publications_with_doi(id_etab, min_year)
    df_scanr.drop(columns = ["title", "year"], inplace = True)
    print(f"nb de DOI récupérés de ScanR \t{len(df_scanr)}")
    #df_scanr.to_csv("data\doi_scanr.csv", columns= ["doi"], index = False)
    
    # concatener et dédoublonner
    df = pd.concat([df_mydois, df_scanr])
    df["doi"] = df["doi"].str.lower() # doi en minuscule
    df = df[ (~df['doi'].duplicated()) & (df["doi"].notna())].copy()
    df.reset_index(drop=True, inplace=True)
    
else : 
    df = df_mydois

In [4]:
# retrait des publications sans DOI
df.dropna(subset=["doi"], inplace = True)

# feedback nb de DOI  et tps de traitement
print(f" nb de DOI a traiter \t{len(df)}")
print(f" /!\ temps estimé ~{round(len(df) * 40/1200)} minutes")

 nb de DOI a traiter 	400
 /!\ temps estimé ~13 minutes


## 2. Enrichir

In [5]:
# 2.1. pour chaque publications déduire la présence dans HAL
df_hal = enrich_w_hal(df) # renseigner df[:50].copy() pour tester uniquement sur les 50 premiers DOI

#retirer ce qui est déjà déposé avec fichier
df_no_file = df_hal[ df_hal["submitType"] != "file" ].copy()
print(f"nb de DOI après retrait de ceux en TI dans HAL {len(df_no_file)}")

nb DOI a verifier dans HAL 400
hal 10% 
hal 20% 
hal 30% 
hal 40% 
hal 50% 
hal 60% 
hal 70% 
hal 80% 
hal 90% 
hal 100%
nb de DOI après retrait de ceux en TI dans HAL 352


In [7]:
# 2.2. pour chaque publications ajouter les informations d'accès ouvert via Unpaywall
df_upw = enrich_w_upw(df_no_file)

nb DOI a verifier dans upw 	352
upw 10% 
upw 20% 
upw 30% 
upw 40% 
upw 50% 
upw 60% 
upw 70% 
upw 80% 
upw 89% 
upw 99% 
upw 100%


In [8]:
# remplacer nan/None par des champs vides
df_upw["oa_repo_link"].replace(np.nan, "", inplace = True)
df_upw["oa_repo_link"].replace("None", "", inplace = True)
df_upw["oa_publisher_license"].replace(np.nan, "", inplace = True)

In [9]:
# 2.3. ajouter les possibilités de dépôt via l'API Permissions
df_upw["deposit_condition"] = df_upw.apply(lambda row : add_permissions(row), axis = 1)

10.1016/j.omtn.2023.03.012 publishedVersion can be shared 🎉
10.1016/j.resplu.2023.100381 publishedVersion can be shared 🎉
10.1016/j.jgar.2023.02.005 publishedVersion can be shared 🎉
10.1016/j.resmer.2022.100981 acceptedVersion , no embargo
10.1530/REP-22-0416 acceptedVersion , no embargo
10.1200/JCO.22.00437 publishedVersion can be shared 🎉
10.1177/19714009221109885 acceptedVersion , no embargo
10.1093/bjd/ljac134 acceptedVersion , no embargo
10.1093/gerona/glac234 acceptedVersion , no embargo
10.1002/14651858.CD015490 acceptedVersion , no embargo
10.1182/blood.2022016943 publishedVersion can be shared 🎉
10.15252/emmm.202216320 publishedVersion can be shared 🎉
doi problem w permissions 10.4244/EIJ-D-22-00723
10.1002/jmd2.12358 publishedVersion can be shared 🎉
10.1093/clinchem/hvac201 acceptedVersion , no embargo
10.1016/j.banm.2022.10.019 acceptedVersion , no embargo
10.1016/j.idnow.2023.01.001 acceptedVersion , no embargo
10.1002/adbi.202200224 acceptedVersion , no embargo
10.1136/tho

In [10]:
## export des données brutes
df_upw.to_csv("data/raw__dois_hal_upw_perm.csv", index = False)

## 3. Trier et Formater

In [11]:
df_final = df_upw
df_final.fillna("", inplace = True) # sinon nan sera compris comme une string non vide

## retirer ce qui est dans HAL qui a un lien extérieur et qui est signalé en repository dans upw
index2remove = df_final[ (df_final["linkExtId"] != "") & (df_final["oa_repo_link"] != "") ].index
df_final.drop(index2remove, inplace = True)
print(f"nb DOI retiré car marqué 'open' dans HAL et repository dans upw {len(index2remove)}" )

nb DOI retiré car marqué 'open' dans HAL et repository dans upw 4


In [12]:
# deduire la colonne todo
df_final["todo"] = df_final.apply(lambda row : deduce_todo(row), axis = 1)

In [13]:
## retrait des colonnes non utilisées
remove_cols = ["submitType", "has_issn"]
for col in remove_cols : 
    if col in df_final.columns :
        del df_final[col]

In [15]:
#imprimer des statistiques brutes
print(f"\nStatistiques\n\nnb de DOI a traiter\t\t\t{len(df_final)}\n{df_final['todo'].value_counts()}")


Statistiques

nb de DOI a traiter			348
ecrire a l auteur pour appliquer la LRN                      144
selon la licence ajouter le PDF editeur                      129
creer ou retrouver la notice                                  63
recuperer le PDF editeur et ecrire a l auteur pour accord      9
verifier les identifiants de la notice                         2
Name: todo, dtype: int64


In [16]:
df_final.to_csv("data\dois_a_traiter.csv", index = False)
df_final.columns

Index(['doi', 'halId', 'linkExtId', 'upw_state', 'published_date',
       'oa_publisher_license', 'oa_publisher_link', 'oa_repo_link',
       'deposit_condition', 'todo'],
      dtype='object')

In [25]:
# rendre la lecture dans libreOffice plus élégante avec l'ajout d'hyperlien
df_final["doi"] = df_final.apply(lambda row : addCaclLinkFormula("https://doi.org/", row["doi"], row["doi"]), axis = 1)  
df_final["halId"] = df_final.apply(lambda row : addCaclLinkFormula("https://hal.archives-ouvertes.fr/", row["halId"], row["halId"]), axis = 1)  
df_final["oa_publisher_link"] = df_final.apply(lambda row : addCaclLinkFormula("", row["oa_publisher_link"], row["oa_publisher_link"]), axis = 1)	
df_final["oa_repo_link"] = df_final.apply(lambda row : addCaclLinkFormula("", row["oa_repo_link"], row["oa_repo_link"]), axis = 1)	

In [26]:
## exporter les données sous forme de tableau pour libreOffice
df_final.to_csv("data\dois_a_traiter_formules.csv", index = False)